# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298379683238                   -0.85    5.0         
  2   -8.300192669659       -2.74       -1.25    1.0    126ms
  3   -8.300432234005       -3.62       -1.89    3.0    167ms
  4   -8.300462008285       -4.53       -2.77    1.6    141ms
  5   -8.300464148889       -5.67       -3.09    3.1    246ms
  6   -8.300464398868       -6.60       -3.28    8.5    255ms
  7   -8.300464520230       -6.92       -3.44    1.8    150ms
  8   -8.300464587007       -7.18       -3.59    2.1    148ms
  9   -8.300464626042       -7.41       -3.77    1.5    150ms
 10   -8.300464637233       -7.95       -3.92    1.5    168ms
 11   -8.300464643593       -8.20       -4.27    2.1    213ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67446158606                   -0.70    7.4         
  2   -16.67864144538       -2.38       -1.14    1.0    341ms
  3   -16.67921044498       -3.24       -1.87    3.0    404ms
  4   -16.67927749637       -4.17       -2.76    3.2    425ms
  5   -16.67928584774       -5.08       -3.18    4.8    509ms
  6   -16.67928619870       -6.45       -3.48    2.4    413ms
  7   -16.67928621808       -7.71       -3.94    2.0    342ms
  8   -16.67928622140       -8.48       -4.56    2.4    376ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32655474709                   -0.56    7.2         
  2   -33.33423248302       -2.11       -1.00    1.2    1.27s
  3   -33.33590608711       -2.78       -1.73    5.5    1.70s
  4   -33.33613182575       -3.65       -2.63    5.4    1.58s
  5   -33.33679744230       -3.18       -2.23    6.4    2.02s
  6   -33.33682636288       -4.54       -2.28    1.6    1.19s
  7   -33.33694323929       -3.93       -3.57    1.5    1.30s
  8   -33.33694344076       -6.70       -3.52    5.6    1.99s
  9   -33.33694377554       -6.48       -4.24    1.6    1.27s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298517421216                   -0.85    5.1         
  2   -8.300275145860       -2.76       -1.59    1.0    104ms
  3   -8.300439129904       -3.79       -2.71    3.6    157ms
  4   -8.300448274987       -5.04       -2.64    3.1    177ms
  5   -8.300464440148       -4.79       -3.36    1.0    102ms
  6   -8.300464610991       -6.77       -3.94    1.8    122ms
  7   -8.300464641376       -7.52       -4.42    2.1    137ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32388459866                   -0.56    7.2         
  2   -33.27773582598   +   -1.34       -1.25    1.0    1.01s
  3   +12.10641776631   +    1.66       -0.24    8.8    2.56s
  4   -33.31954601976        1.66       -1.68    6.4    2.36s
  5   -33.21794538304   +   -0.99       -1.29    4.1    1.81s
  6   -32.62415164519   +   -0.23       -1.14    3.8    1.66s
  7   -33.29109313865       -0.18       -1.71    5.1    1.68s
  8   -33.33451146337       -1.36       -2.14    2.4    1.20s
  9   -33.33455647378       -4.35       -2.09    3.0    1.55s
 10   -33.33676424903       -2.66       -2.78    1.8    1.20s
 11   -33.33693312319       -3.77       -3.01    3.5    1.53s
 12   -33.33693681293       -5.43      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.